In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Text, \
    select, update, func
from tqdm.notebook import tqdm
import praw
import time

In [2]:
db_name = 'legaladvice.db'
subreddit = 'legaladvice'
seed = 0
stop_at = 100000
exclude_top_level = True

In [3]:
engine = create_engine('sqlite:///%s' % db_name, echo = False)
meta = MetaData()

submissions_table = Table(
   'submissions', meta, 
    Column('id', String, primary_key=True),
    Column('status', Integer),
)

In [4]:
meta = MetaData()
comments_table = Table(
   'comments', meta, 
    Column('comment_id', String, primary_key=True),
    Column('submission_id', String),
    Column('url', String),
    Column('comment', Text),    
)
meta.create_all(engine)

In [5]:
reddit = praw.Reddit(
    client_id="PTYaeRF8ATlqboPxx-3u6g",
    client_secret="o9GvtpP9xk-CXfj7Y5Tys62dOZSyRw",
    user_agent="comment extraction for prevalence estimation (by u/patroklos1)",
    #password="8PmSSKqNfnQ3hvy",
    #username="patroklos1",
)

In [ ]:
while True:
    with engine.connect() as conn:
        num_rows = engine.execute('select count(comment_id) from comments').scalar()
        
    print(num_rows)
    if num_rows >= stop_at:
        break
    
    with engine.connect() as conn:
        # get an unvisited submission
        stmt = submissions_table.select()
        stmt = stmt.where(submissions_table.c.status==0).order_by(func.random()).limit(1)
        row = next(conn.execute(stmt))
        submission_id, status = row

        # update
        stmt = submissions_table.update()
        stmt = stmt.where(submissions_table.c.id==submission_id)
        stmt = stmt.values(status=1)
        conn.execute(stmt)

    # retrieve comments
    submission = reddit.submission(submission_id)
    submission.comments.replace_more(limit=None)
    data = []
    for comment in submission.comments.list():
        url = "https://www.reddit.com/r/%s/comments/%s/comment/%s" % (subreddit, submission_id, comment.id)
        
        if exclude_top_level and comment.parent_id.startswith('t3'):
            continue
        
        data.append((comment.id, submission_id, url, comment.body))

    # save
    with engine.connect() as conn:
        # update
        stmt = submissions_table.update()
        stmt = stmt.where(submissions_table.c.id==submission_id)
        stmt = stmt.values(status=2)
        conn.execute(stmt)

        if len(data) == 0:
            continue
        else:
            for id, submission_id, url, comment in data:
                ins = comments_table.insert().values(
                    comment_id=id, submission_id=submission_id, url=url, comment=comment)
                result = conn.execute(ins)
        
    # time.sleep(0.1)

/tmp/ipykernel_33867/589220735.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  num_rows = engine.execute('select count(comment_id) from comments').scalar()


0
0
2
3
3
3
3
10
10


In [ ]:
with engine.connect() as conn:
    stmt = comments_table.select()
    stmt = stmt.order_by(func.random()).limit(1)
    row = next(conn.execute(stmt))
    print(row)

In [ ]:
with engine.connect() as conn:
    total_rows = engine.execute('select count(id) from submissions').scalar()
    print(total_rows)
    collected_rows = engine.execute('select count(id) from submissions where status=0').scalar()
    print(collected_rows)
    
    print(1 - collected_rows / total_rows)